# Stuart Landau Example 

This notebook contains the 2D-Stuart-Landau example from the MCBB paper that varies $r_2$. For all details please see the paper. The other Stuart Landau configurations are in seperate files

For loading the saved jld files use the branch `old-state-sl-paper` of MCBB. 

In [1]:
cluster = false
calc_distance = false 
using Distributed
if cluster
    using ClusterManagers
    N_tasks = parse(Int, ARGS[1])
    task_type = parse(Int, ARGS[2])
    SAVE_NAME = ARGS[3]

    if task_type == 1
        N_worker = N_tasks
        addprocs(SlurmManager(N_worker))
        calc_mc = true 
    elseif task_type == 2
        calc_mc = false
        calc_distance = true
    else
        error("Unkown task_type, 1: Solve MCBB Problem, 2: Compute Distances")
    end
else
    SAVE_NAME = "stuart-landau-standard-multi-r2"
    using Plots
    import PyPlot, GraphPlot
end

@everywhere using MCBB
using JLD2, Clustering, LinearAlgebra, SparseArrays
@everywhere using Parameters, Distributions, DifferentialEquations, StatsBase, LightGraphs





@everywhere struct stuart_landau_sathiyadevi_pars <: DEParameters
    lambda::Real
    omega::Real
    eps::Real
    N::Int
    P_1::Int
    P_2::Int
    A_1::AbstractArray
    A_2::AbstractArray
end

@everywhere function stuart_landau_sathiyadevi_pars(lambda::Real, omega::Real, eps::Real, N::Int, P_1::Int, P_2::Int, p_r::Number=0.)
    A_1 = Bool.(adjacency_matrix(watts_strogatz(N, 2*P_1, p_r)))
    A_2 = Bool.(adjacency_matrix(watts_strogatz(N, 2*P_2, p_r)))
    stuart_landau_sathiyadevi_pars(lambda, omega, eps, N, P_1, P_2, A_1, A_2)
end

@everywhere function stuart_landau_sathiyadevi_pars(lambda::Real, omega::Real, eps::Real, N::Int, r_1::Real, r_2::Real)
    P_1 = Int(round(r_1*N))
    P_2 = Int(round(r_2*N))
    stuart_landau_sathiyadevi_pars(lambda, omega, eps, N, P_1, P_2)
end


@everywhere function stuart_landau_sathiyadevi!(dz,z,p::stuart_landau_sathiyadevi_pars,t)
    epsP1 = p.eps/(2*p.P_1)
    epsP2 = p.eps/(2*p.P_2)

    for i=1:p.N
        attr_coup = 0.
        for j=1:N
            if p.A_1[i,j]
               attr_coup += complex(real(z[j]) - real(z[i]),0)
            end
        end
        attr_coup *= epsP1

        repul_coup = 0.
        for j=1:p.N
            if p.A_2[i,j]
                repul_coup += complex(0., imag(z[j]) - imag(z[i]))
            end
        end
        repul_coup *= epsP2

        dz[i] = (complex(p.lambda,p.omega) - abs(z[i])^2)*z[i] + attr_coup - repul_coup
    end
end


@everywhere N = 100
@everywhere r_1 = 0.01
@everywhere r_2 = 0.22
@everywhere omega = 2.0
@everywhere lambda = 1.0
@everywhere epsval = 2.02
@everywhere pars = stuart_landau_sathiyadevi_pars(lambda, omega, epsval, N, r_1, r_2)

@everywhere epsilons = () -> rand(Uniform(1.7,2.5))
@everywhere p_r_gen = () -> rand(Uniform(0.,0.1))
@everywhere r_2_gen = () -> rand(Uniform(0.01,0.24))
@everywhere ic_dist = () -> complex(rand(Uniform(-1,1)),rand(Uniform(-1,1)))

@everywhere N_ics = 18000
@everywhere tail_frac = 0.7

#-@everywhere par_vars = (:eps, epsilons)

@everywhere new_sls_pars(old_par::stuart_landau_sathiyadevi_pars; eps=2.02, r_2_val=0.22) = stuart_landau_sathiyadevi_pars(old_par.lambda, old_par.omega, eps, old_par.N, r_1, r_2_val)
@everywhere par_vars = MultiDimParameterVar([OneDimParameterVar(:eps, epsilons),OneDimParameterVar(:r_2_val,r_2_gen)], new_sls_pars)

@everywhere odeprob = ODEProblem(stuart_landau_sathiyadevi!, rand(Complex{Float64},N), (0.,200.), pars)


@everywhere mean_real(arr::Array{Complex{Float64},1}, past_measures::AbstractArray) = mean(real.(arr))
@everywhere std_real(arr::Array{Complex{Float64},1}, past_measures::AbstractArray) = std(real.(arr); mean=past_measures[1])
@everywhere kl_real(arr::AbstractArray, past_measures::AbstractArray) = empirical_1D_KL_divergence_hist(real.(arr), past_measures)

@everywhere mean_imag(arr::Array{Complex{Float64},1}, past_measures::AbstractArray) = mean(imag.(arr))
@everywhere std_imag(arr::Array{Complex{Float64},1}, past_measures::AbstractArray) = std(imag.(arr); mean=past_measures[4])
@everywhere kl_imag(arr::AbstractArray, past_measures::AbstractArray) = empirical_1D_KL_divergence_hist(imag.(arr), [past_measures[4], past_measures[5]])

@everywhere std_max(arr::AbstractArray) = maximum(std(real.(arr)))

@everywhere function eval_ode_run_stuart(sol,i)
    (N_dim, __) = size(sol)
    state_filter = collect(1:N)
    eval_funcs = [mean_real, std_real, kl_real, mean_imag, std_imag, kl_imag]
    global_eval_funcs = [std_max]
    matrix_eval_funcs = [correlation_ecdf]
    eval_ode_run(sol, i, state_filter, eval_funcs, matrix_eval_funcs, global_eval_funcs, flag_past_measures=true)
end



if cluster
    prob = DEMCBBProblem(odeprob, ic_dist, N_ics, pars, par_vars, eval_ode_run_stuart, tail_frac);
    println("solving now...")
    sol = solve(prob);
    println("finished!")
    @save string(SAVE_NAME,"-ps.jld2") prob sol
else
    @load string(SAVE_NAME,"-ps.jld2") prob sol
end

weights = [0.25, 1., 0., 0.25, 1., 0., 0., 0.5, 1.,0.5]
distance_func = (x,y)->sum(abs.(x.-y))

K_BIN_FAC = 0.25

if calc_distance
    sol = normalize(sol)

    #println("calculating first distance matrix")
    #D = distance_matrix(sol, prob, [0.5, 1., 0.25, 0.5, 1., 0.25, 1.])
    #@save string(SAVE_NAME,"-d.jld2") D

    println("calculating distance matrix")
    
    D_hist = distance_matrix(sol, prob, distance_func, weights, histograms=true, matrix_distance_func=MCBB.wasserstein_histogram_distance, k_bin=K_BIN_FAC)
    @save string(SAVE_NAME,"-d-hist-corr.jld2") D_hist
    
    # normalize sol for the distance computation but not for the later plots and so on. 
    @load string(SAVE_NAME,"-ps.jld2") prob sol
else
    if !(cluster)
        #@load string(SAVE_NAME,"-d.jld2") D
        @load string(SAVE_NAME,"-d-hist-corr.jld2") D_hist
    end
end

┌ Warning: read type ODEFunction{true,typeof(stuart_landau_sathiyadevi!),UniformScaling{Bool},Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,TCV} where TCV is not a leaf type in workspace; reconstructing
└ @ JLD2 /home/max/.julia/packages/JLD2/KjBIK/src/data.jl:477
┌ Warning: some parameters could not be resolved for type EnsembleProblem{ODEProblem{Array{Complex{Float64},1},Tuple{Float64,Float64},true,stuart_landau_sathiyadevi_pars,getfield(JLD2.ReconstructedTypes, Symbol("##ODEFunction{true,typeof(stuart_landau_sathiyadevi!),UniformScaling{Bool},Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,TCV} where TCV#363")),Nothing,DiffEqBase.StandardODEProblem},getfield(MCBB, Symbol("#new_problem#37")){Array{Complex{Float64},2},Array{Float64,2},stuart_landau_sathiyadevi_pars,Int64,Array{getfield(MCBB, Symbol("##39#40")){getfield(Main, Symbol("##9#10"))},1},MultiDimParameterVarFunc},typeof(eval_ode_run_stuart),DiffEqBase.##388#394,Array{Any,1}}; reconstructing
└

MethodError: MethodError: Cannot `convert` an object of type getfield(JLD2.ReconstructedTypes, Symbol("##EnsembleProblem{ODEProblem{Array{Complex{Float64},1},Tuple{Float64,Float64},true,stuart_landau_sathiyadevi_pars,getfield(JLD2.ReconstructedTypes, Symbol("##ODEFunction{true,typeof(stuart_landau_sathiyadevi!),UniformScaling{Bool},Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,TCV} where TCV#363")),Nothing,DiffEqBase.StandardODEProblem},getfield(MCBB, Symbol("#new_problem#37")){Array{Complex{Float64},2},Array{Float64,2},stuart_landau_sathiyadevi_pars,Int64,Array{getfield(MCBB, Symbol("##39#40")){getfield(Main, Symbol("##9#10"))},1},MultiDimParameterVarFunc},typeof(eval_ode_run_stuart),DiffEqBase.##388#394,Array{Any,1}}#364")) to an object of type EnsembleProblem
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T at essentials.jl:154
  EnsembleProblem(::T, !Matched::T2, !Matched::T3, !Matched::T4, !Matched::T5) where {T, T2, T3, T4, T5} at /home/max/.jul

In [2]:
k = 4
fdist2 = k_dist(D_hist,k);

UndefVarError: UndefVarError: D_hist not defined

In [3]:
x = collect(1:sol.N_mc)
plot(x[1:end],fdist2[1:end])

UndefVarError: UndefVarError: sol not defined

In [4]:
db_eps = 0.07
#db_res_hist = dbscan(D_hist,db_eps,20)
db_res_hist = dbscan(D_hist,db_eps,20)

db_res_hist.counts

UndefVarError: UndefVarError: D_hist not defined

In [5]:
cluster_n_noise(db_res_hist)

UndefVarError: UndefVarError: db_res_hist not defined

In [6]:
cluster_members = cluster_membership(prob, db_res_hist, [0.05,0.01], [0.01,0.0025]);


UndefVarError: UndefVarError: prob not defined

In [7]:
clabels = ["Outlier", "OD+PS", "TW", "DS", "IW2","RCSW"]

6-element Array{String,1}:
 "Outlier"
 "OD+PS"  
 "TW"     
 "DS"     
 "IW2"    
 "RCSW"   

In [8]:
using LaTeXStrings
Plots.pyplot()


l = @layout [a; b; c; d; e]
p1 = Plots.plot(cluster_members, plot_index=1, xlabel=L"Coupling $K$", ylabel=L"$r_2$", title=string(clabels[1], " Cluster"), colorbar_title="rel. Size of Basin")
p2 = Plots.plot(cluster_members, plot_index=2, xlabel=L"Coupling $K$", ylabel=L"$r_2$", title=string(clabels[2], " Cluster"), colorbar_title="rel. Size of Basin")
p3 = Plots.plot(cluster_members, plot_index=3, xlabel=L"Coupling $K$", ylabel=L"$r_2$", title=string(clabels[3], " Cluster"), colorbar_title="rel. Size of Basin")
p4 = Plots.plot(cluster_members, plot_index=4, xlabel=L"Coupling $K$", ylabel=L"$r_2$", title=string(clabels[4], " Cluster"), colorbar_title="rel. Size of Basin")
p5 = Plots.plot(cluster_members, plot_index=6, xlabel=L"Coupling $K$", ylabel=L"$r_2$", title=string(clabels[6], " Cluster"), colorbar_title="rel. Size of Basin")


Plots.plot([p1, p2, p3, p4, p5]..., size=(400,1400), layout=l)

UndefVarError: UndefVarError: cluster_members not defined

In [9]:
Plots.savefig("sl-2d-r2-cm.pdf")

ErrorException: No current plot/subplot

In [10]:
colororder = ["blue","orange","green","red","purple","brown","magenta","olive","cyan"]
measure_1 = 1
measure_2 = 2

m1m = cluster_measure_mean(sol, db_res_hist, measure_1)
m1sd = cluster_measure_std(sol, db_res_hist, measure_1)
m2m = cluster_measure_mean(sol, db_res_hist, measure_2)
m2sd = cluster_measure_std(sol, db_res_hist, measure_2)
plot_i = 1:8
SC = PyPlot.scatter(m1m[plot_i], m2m[plot_i], c=colororder[plot_i])
#PyPlot.legend(SC,["Noise","2","3","4","5","6","7","8","9","10","11"])

PyPlot.errorbar(m1m[plot_i], m2m[plot_i], fmt="o", ms=0, ecolor=colororder, xerr=0.005*m1sd[plot_i], yerr=0.1*m2sd[plot_i])
PyPlot.xlabel("Average Mean")
PyPlot.ylabel("Average Std")

UndefVarError: UndefVarError: sol not defined

In [11]:
traj1 = get_trajectory(prob, sol, db_res_hist, 1)
IM = PyPlot.imshow(Matrix(imag.(traj1)))
PyPlot.ylabel(L"Oscillator $i$",fontsize=20) 
PyPlot.xlabel(L"Time $t$",fontsize=20)
PyPlot.title(string("Example Trajectory - ",clabels[1]), fontsize=20)
cb = PyPlot.colorbar(IM, orientation="horizontal")
cb[:ax][:set_ylabel]("Im(z)", fontsize=20)


UndefVarError: UndefVarError: prob not defined

In [12]:
traj2 = get_trajectory(prob, sol, db_res_hist, 2)
IM = PyPlot.imshow(Matrix(imag.(traj2)))
PyPlot.ylabel(L"Oscillator $i$",fontsize=20) 
PyPlot.xlabel(L"Time $t$",fontsize=20)
PyPlot.title(string("Example Trajectory - ",clabels[2]), fontsize=20)
cb = PyPlot.colorbar(IM, orientation="horizontal")
cb[:ax][:set_ylabel]("Im(z)", fontsize=20)

UndefVarError: UndefVarError: prob not defined

In [13]:
traj3 = get_trajectory(prob, sol, db_res_hist, 3)
IM = PyPlot.imshow(Matrix(imag.(traj3)))
PyPlot.ylabel(L"Oscillator $i$",fontsize=20) 
PyPlot.xlabel(L"Time $t$",fontsize=20)
PyPlot.title(string("Example Trajectory - ",clabels[3]), fontsize=20)
cb = PyPlot.colorbar(IM, orientation="horizontal")
cb[:ax][:set_ylabel]("Im(z)", fontsize=20)

UndefVarError: UndefVarError: prob not defined

In [14]:
traj4 = get_trajectory(prob, sol, db_res_hist, 4)
IM = PyPlot.imshow(Matrix(imag.(traj4)))
PyPlot.ylabel(L"Oscillator $i$",fontsize=20) 
PyPlot.xlabel(L"Time $t$",fontsize=20)
PyPlot.title(string("Example Trajectory - ",clabels[4]), fontsize=20)
cb = PyPlot.colorbar(IM, orientation="horizontal")
cb[:ax][:set_ylabel]("Im(z)", fontsize=20)

UndefVarError: UndefVarError: prob not defined

In [15]:
traj6 = get_trajectory(prob, sol, db_res_hist, 6)
IM = PyPlot.imshow(Matrix(imag.(traj6)))
PyPlot.ylabel(L"Oscillator $i$",fontsize=20) 
PyPlot.xlabel(L"Time $t$",fontsize=20)
PyPlot.title(string("Example Trajectory - ",clabels[6]), fontsize=20)
cb = PyPlot.colorbar(IM, orientation="horizontal")
cb[:ax][:set_ylabel]("Im(z)", fontsize=20)

UndefVarError: UndefVarError: prob not defined